## Question 1: **Answer**

In [1]:
import pandas as pd
import sqlite3

In [ ]:
def add_data_to_database(input_data):
    try:
        # Creating a DataFrame
        data = pd.read_csv(input_data)
        # Creating a database
        engine = sqlite3.connect('bakery_records.db')
        data.to_sql('bakery', con=engine, index = False)
        engine.close()
    except Exception as e:
        print(f"Error:{e}")

data = "/content/bakery_business.csv"
add_data_to_database(data)

In [3]:
# Loading SQL extension
%load_ext sql

In [4]:
#  Connecting to the database
%sql sqlite:///bakery_records.db

In [5]:
# Default to duplicated style
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [6]:
%%sql
SELECT * FROM bakery
LIMIT 5;

 * sqlite:///bakery_records.db
Done.


order_id,order_date,customer_name,order_items,order_total
1,2020-10-05,Alice Smith,Cake,25
2,2020-10-06,Bob Johnson,Chocolate Chip Cookies,12
3,2020-10-07,Charlie Brown,Strawberry Cupcakes,18
4,2020-10-08,David Williams,Vanilla Muffin,15
5,2020-10-09,Emily Jones,Chocolate Cake,30


## Question 2: **Answer**

In [7]:
%%sql
SELECT
    order_date,
    customer_name,
    order_items
FROM bakery
WHERE customer_name = 'Laura Brown' AND
    order_date = '2020-10-19';

 * sqlite:///bakery_records.db
Done.


order_date,customer_name,order_items
2020-10-19,Laura Brown,Chocolate Chip Cookies


## Question 3: **Answer**

In [8]:
%%sql
SELECT
    SUM(order_total) AS total_from_three_customers,
    ROUND(SUM(order_total)/CAST((SELECT SUM(order_total)
                                 FROM bakery)
                                 AS REAL), 2) * 100 AS percentage
FROM bakery
WHERE customer_name IN ('Charlie Brown', 'Bob Johnson', 'Alice Smith')
    AND order_date BETWEEN '2020- 10-01' AND '2020-10-31';

 * sqlite:///bakery_records.db
Done.


total_from_three_customers,percentage
55,18.0


## Question 4: **Answer**

In [9]:
%%sql
SELECT
    SUM(total) AS total_number
FROM (SELECT COUNT(*) AS total
      FROM bakery
      WHERE order_total IN (10, 12)
      GROUP BY order_total)
      AS subquery;

 * sqlite:///bakery_records.db
Done.


total_number
5


## **Part B**

In [10]:
%%sql
-- CTE to count orders at price 10 and 12
WITH count_orders AS (
    SELECT
        order_total,
        COUNT(*) AS total
    FROM bakery
    WHERE order_total IN (10, 12)
    GROUP BY order_total
)
-- main query to return total number of orders
SELECT
    SUM(total) AS total_number
FROM count_orders;

 * sqlite:///bakery_records.db
Done.


total_number
5


## Question 5: **Answer**

In [11]:
%%sql
SELECT
 -- Calculate the percentage for the first half october
    ROUND(
        (SUM(CASE WHEN order_date BETWEEN '2020-10-01' AND '2020-10-15'
              THEN order_total ELSE 0 END)
        * 100.0)
        / COALESCE(SUM(order_total), 1), 2)
        AS first_half_october_percentage,

    -- Calculate the percentage for the second half october
    ROUND(
        (SUM(CASE WHEN order_date BETWEEN '2020-10-16' AND '2020-10-31'
              THEN order_total ELSE 0 END)
        * 100.0)
        / COALESCE(SUM(order_total), 1), 2)
        AS second_half_october_percentage
FROM bakery
WHERE order_date BETWEEN '2020-10-01' AND '2020-10-31';

 * sqlite:///bakery_records.db
Done.


first_half_october_percentage,second_half_october_percentage
53.51,46.49
